In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import os
import wandb
from wandb.keras import WandbCallback

In [5]:
def train():
    import pandas as pd
    import tensorflow as tf
    from tensorflow import keras
    import matplotlib.pyplot as plt
    import numpy as np
    import wandb
    from wandb.keras import WandbCallback

    def seq_acc(y_true, y_pred):
        y_bin=np.zeros_like(y_pred)
        for i, dd in enumerate(y_bin):
            for j in range(len(dd)):
                pred=y_pred[i][j]
                if pred>=0.5:
                    y_bin[i][j]=1
                else:
                    y_bin[i][j]=0

        predict_true = (y_true == y_bin)
        # multi-column일 경우에도 계산 할 수 있도록 np.average를 한번 더 씌움
        try:
            score = np.average(np.average(predict_true))
        except ValueError:
            score = mean_squared_error(y_true, y_bin)
        return score

    def my_seq_acc(y_true, y_pred):
        score = tf.py_function(func=seq_acc, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_seq_acc') # tf 2.x
        #score = tf.py_func( lambda y_true, y_pred : mse_AIFrenz(y_true, y_pred) , [y_true, y_pred], 'float32', stateful = False, name = 'custom_mse' ) # tf 1.x
        return score


    class MySeqAccCallback(keras.callbacks.Callback):
        def on_epoch_end(self, epochs, logs=None):
            y_pred=self.model.predict(X_test)
            print('sequence accuracy is {}'.format(seq_acc(y_test, y_pred)))


    default_config={
                         'seq_field':72,
                         'stride_inside_seq':9,
                         'stride_between_seqs':2,
                         'learning_rate':0.01,
                         'split_train_ratio':0.8,
                         'epochs':20,
                         'batch_size':64,
                         'unit_LSTM0':64}
    wandb.init(config = default_config)

    locations=['거문도', '울산', '거제도', '통영', '추자도']

    # load normalized data

    df_merged=pd.read_csv("sensory_preprocessed_df_condition_labeled.csv")
    if df_merged.columns[0]=='Unnamed: 0':
        df_merged = df_merged.iloc[:, 1:]

    print('loaded dataset. Generating sequences')
    seq_length=wandb.config.seq_field//wandb.config.stride_inside_seq
    len_ds=len(df_merged)

    seqs_idx=[]

    start_idx=0
    while start_idx<=len_ds-wandb.config.seq_field:
        seqs_idx.append(list(range(start_idx, start_idx + wandb.config.seq_field, wandb.config.stride_inside_seq
    )))
        start_idx+=wandb.config.stride_between_seqs


    seqs_idx[100],len(seqs_idx[100])

    df_merged.reset_index(inplace=True, drop=True)
    print('Any missing values exist:', df_merged.isna().all().all())


    #train_cols=['풍속(m/s)', '풍향(deg)', '기온(°C)', '수온(°C)', '강수량(mm)', '적조발생']
    ds_train_cols=df_merged
    ds_train_cols.reset_index(inplace=True, drop=True)
    print('train dataset columns:',ds_train_cols.columns)

    seq_dataset=np.zeros([len(seqs_idx), len(seqs_idx[0]), len(ds_train_cols.columns)])

    for i, seq in enumerate(seqs_idx):
        for j, row_number in enumerate(seq):
            seq_dataset[i, j]=ds_train_cols.loc[row_number].to_numpy()

    def not_bin_in_occurence(x):
        if x==1 or x==0:
            return x
        else:
            print('exceptional value(not 0 or 1) found. replaced by near one.')
            if x>=0.5:
                return 1
            else:
                return 0
            
    ds_train_cols['적조발생(조건)']=ds_train_cols['적조발생(조건)'].apply(not_bin_in_occurence)


    split_index=int(len(seq_dataset)*wandb.config.split_train_ratio)
    print(split_index, len(seq_dataset))

    train_xy=seq_dataset[:split_index]
    np.random.shuffle(train_xy)
    X_train=train_xy[:,:,0:-1]
    y_train=train_xy[:,:,-1]

    test_xy=seq_dataset[split_index:]
    np.random.shuffle(test_xy)
    X_test=test_xy[:,:,0:-1]
    y_test=test_xy[:,:,-1]

    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape,'\n\n')

    model = keras.Sequential([
        keras.Input(shape=(seq_length, 25)),
        keras.layers.LSTM(wandb.config.unit_LSTM0),
        keras.layers.Dense(seq_length, activation="sigmoid"),
    ]
    )


    # In[95]:
    optimizer=keras.optimizers.Adam(learning_rate=wandb.config.learning_rate)

    model.compile(optimizer=optimizer, loss="binary_crossentropy")

    # In[97]:


    # In[98]:


    model.fit(X_train, y_train,
            batch_size=wandb.config.batch_size,
            epochs=wandb.config.epochs, 
            validation_data=(X_test, y_test),
            callbacks=[WandbCallback(training_data = (X_train, y_train),
                                     validation_data = (X_test, y_test)), MySeqAccCallback()])
    y_pred=model.predict(X_test)
    wandb.log({"Validation Acc :":seq_acc(y_test, y_pred)})

In [6]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc


True

In [7]:
sweep_config = {
    "name": "sweep",
    "metric": {
        "goal": "minimize",
        "name": "val_loss"
    },
    "method": "random",
    "parameters": {
        "split_train_ratio":{
            "values":[0.5,0.8]
        },
        "epochs":{
            "values":[1,2,3,4,8,12]
        },
        "learning_rate":{"values":[0.0002, 0.001, 0.0032, 0.01]},
        "unit_LSTM0":{"values":[32,64,128]},
        "stride_inside_seq":{"values":[3,9]}
    }
    }

In [9]:
sweep_id = wandb.sweep(sweep_config)

# run the sweep
wandb.agent(sweep_id,
            function=train,
            entity = 'chhyyi',
            project = 'redzone_LSTM_sweep_condition_labeled')

Create sweep with ID: o7gddbz8
Sweep URL: https://wandb.ai/chhyyi/redzone_LSTM_sweep_condition_labeled/sweeps/o7gddbz8


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: a0ovrp0e with config:
wandb: 	epochs: 12
wandb: 	learning_rate: 0.0002
wandb: 	split_train_ratio: 0.8
wandb: 	stride_inside_seq: 3
wandb: 	unit_LSTM0: 32
wandb: Currently logged in as: chhyyi. Use `wandb login --relogin` to force relogin


loaded dataset. Generating sequences
Any missing values exist: False
train dataset columns: Index(['풍속(m/s)', '풍향(deg)', '기온(°C)', '수온(°C)', '강수량(mm)', '풍속(m/s).1',
       '풍향(deg).1', '기온(°C).1', '수온(°C).1', '강수량(mm).1', '풍속(m/s).2',
       '풍향(deg).2', '기온(°C).2', '수온(°C).2', '강수량(mm).2', '풍속(m/s).3',
       '풍향(deg).3', '기온(°C).3', '수온(°C).3', '강수량(mm).3', '풍속(m/s).4',
       '풍향(deg).4', '기온(°C).4', '수온(°C).4', '강수량(mm).4', '적조발생(조건)'],
      dtype='object')
20106 25133
(20106, 24, 25) (5027, 24, 25) (20106, 24) (5027, 24) 




wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/12
315/315 [==============================] - 25s 5ms/step - loss: 0.3920 - val_loss: 0.0926


...

INFO:tensorflow:Assets written to: /aiffel/aiffel/aiffelthon/wandb/run-20221113_150655-bybqfkcg/files/model-best/assets


INFO:tensorflow:Assets written to: /aiffel/aiffel/aiffelthon/wandb/run-20221113_150655-bybqfkcg/files/model-best/assets
wandb: Adding directory to artifact (/aiffel/aiffel/aiffelthon/wandb/run-20221113_150655-bybqfkcg/files/model-best)... Done. 0.0s


sequence accuracy is 0.9981698098193682
Epoch 6/12
197/197 [==============================] - 1s 5ms/step - loss: 0.0144 - val_loss: 0.0124
sequence accuracy is 0.9980802896474894
Epoch 7/12
197/197 [==============================] - 1s 5ms/step - loss: 0.0140 - val_loss: 0.0128
sequence accuracy is 0.9981698098193682
Epoch 8/12
197/197 [==============================] - 1s 5ms/step - loss: 0.0125 - val_loss: 0.0137
sequence accuracy is 0.9978050980080104
Epoch 9/12
197/197 [==============================] - 1s 5ms/step - loss: 0.0119 - val_loss: 0.0139
sequence accuracy is 0.998053765152118
Epoch 10/12
197/197 [==============================] - 1s 5ms/step - loss: 0.0112 - val_loss: 0.0134
sequence accuracy is 0.9978913026179677
Epoch 11/12
197/197 [==============================] - 1s 5ms/step - loss: 0.0109 - val_loss: 0.0151
sequence accuracy is 0.9981034985809395
Epoch 12/12
197/197 [==============================] - 1s 5ms/step - loss: 0.0106 - val_loss: 0.0152
sequence accuracy 

Validation Acc :,▁
epoch,▁▂▂▃▄▄▅▅▆▇▇█
loss,█▂▂▂▁▁▁▁▁▁▁▁
val_loss,▅▂▂▂▁▂▃▅▅▄██
Validation Acc :,0.99772
best_epoch,4
best_val_loss,0.01172
epoch,11
loss,0.01063
val_loss,0.01525


wandb: Agent Starting Run: 8k5rlg5b with config:
wandb: 	epochs: 2
wandb: 	learning_rate: 0.0032
wandb: 	split_train_ratio: 0.8
wandb: 	stride_inside_seq: 3
wandb: 	unit_LSTM0: 32
wandb: Ctrl + C detected. Stopping sweep.


In [10]:
wandb.finish()